In [34]:
import numpy as np
import pandas as pd

In [35]:
# 데이터 셋 로딩
pop = pd.read_csv("data/state-population.csv")
areas = pd.read_csv("data/state-areas.csv")
abbrevs = pd.read_csv("data/state-abbrevs.csv") # 축약

In [36]:
print(pop.head());
print(areas.head());
print(abbrevs.head());

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [74]:
#목표 : 2010년 인구 밀도 기준으로 미국 주와 지역 순위를 계산하자.
merged = pd.merge(pop, abbrevs, how = "outer",
                 left_on = "state/region", right_on = "abbreviation")
merged = merged.drop('abbreviation', axis="columns") # 중복 삭제 (state/region, abbreviation)
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


merged

In [52]:
merged.isnull().any() # 맵핑이 안된 값들을 찾는다. 혹은 존재하지 않는 값

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [60]:
merged[merged['population'].isnull()].head()
#2010년전 PR의 데이터에 인구가 없다는 것을 알 수 있음, state 가 null인 것은 맵핑 값이 없음을 의미|

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [56]:
merged.loc[merged['state'].isnull(), 'state/region'].unique() #loc예제 한번더 살펴보기

array(['PR', 'USA'], dtype=object)

In [79]:
##인구 데이터 푸에르로리코(PR)과 전체 미국(USA)에 대한 항목을 초함하고 있지만, 약어키에는 존재하지 않는 것을 알 수 있다.
##약어키에 푸에르로리코와 미국을 추가한다.
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United Sates'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [110]:
# 면적데이터를 추가한다.
final = pd.merge(merged, areas, on="state", how="left")
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [111]:
final.isnull().any() # 누락된 데이터를 찾는다

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [121]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United Sates'], dtype=object)

In [107]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [108]:
data2010 = final.query("year == 2010 & ages == 'total'") #Query the columns of a frame with a boolean expression.
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [109]:
#인구 밀도를 계산하고 순서대로 표시
# 주 기준으로 데이터 인덱스를 재배열
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

                     state/region   ages  year  population  area (sq. mi)
state                                                                    
Alabama                        AL  total  2010   4785570.0        52423.0
Alaska                         AK  total  2010    713868.0       656425.0
Arizona                        AZ  total  2010   6408790.0       114006.0
Arkansas                       AR  total  2010   2922280.0        53182.0
California                     CA  total  2010  37333601.0       163707.0
Colorado                       CO  total  2010   5048196.0       104100.0
Connecticut                    CT  total  2010   3579210.0         5544.0
Delaware                       DE  total  2010    899711.0         1954.0
District of Columbia           DC  total  2010    605125.0           68.0
Florida                        FL  total  2010  18846054.0        65758.0
Georgia                        GA  total  2010   9713248.0        59441.0
Hawaii                         HI  tot

In [31]:
density.head()

state
Alabama        91.287603
Alaska          1.087509
Arizona        56.214497
Arkansas       54.948667
California    228.051342
dtype: float64

In [19]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64